In [1]:
pip install deap


Note: you may need to restart the kernel to use updated packages.


In [41]:
import random
from deap import base, creator, tools, algorithms

def read_graph(input_file):
    with open(input_file, 'r') as file:
        first_line = file.readline().strip().split()
        if len(first_line) != 3 or not all(part.isdigit() for part in first_line):
            raise ValueError(f"Expected the first line to be three numbers, but got: '{' '.join(first_line)}'")
        V, num_interference_edges, num_affinity_edges = map(int, first_line)
        E = set()
        for _ in range(num_interference_edges):
            line = file.readline().strip()
            try:
                u, v = map(int, line.split())
                E.add((u, v))
            except ValueError:
                raise ValueError(f"Invalid format for interference edge: '{line}'")
        A = set()
        for _ in range(num_affinity_edges):
            line = file.readline().strip()
            try:
                u, v = map(int, line.strip().split())
                A.add((u, v))
            except ValueError:
                raise ValueError(f"Invalid format for affinity edge: '{line}'")

    return V, E, A

def eval_graph_coloring(individual, E, A):
    num_colors = len(set(individual))
    satisfied_affinity = sum(individual[x - 1] == individual[y - 1] for x, y in A)
    for u, v in E:
        if individual[u - 1] == individual[v - 1]:
            return 999999, 0  
    return num_colors, -satisfied_affinity
def crossover(ind1, ind2):
    
    size = min(len(ind1), len(ind2))
    cxpoint = random.randint(1, size - 1)
    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]
    return ind1, ind2

def mutate(individual, max_color):
    vertex = random.randrange(len(individual))
    individual[vertex] = random.randint(1, max_color)
    return individual,

def run_nsga2(input_file, output_file, num_generations=100, pop_size=100):
    V, E, A = read_graph(input_file)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    toolbox.register("attr_color", random.randint, 1, V)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_color, n=V)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", eval_graph_coloring, E=E, A=A)
    toolbox.register("mate", crossover)
    toolbox.register("mutate", mutate, max_color=V)
    toolbox.register("select", tools.selNSGA2)
    pop = toolbox.population(n=pop_size)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg1", lambda x: sum(f[0] for f in x) / len(x) if len(x) > 0 else 0)
    stats.register("avg2", lambda x: sum(f[1] for f in x) / len(x) if len(x) > 0 else 0)
    stats.register("min1", lambda x: min(f[0] for f in x) if x else float("inf"))
    stats.register("min2", lambda x: min(f[1] for f in x) if x else float("inf"))
    stats.register("max1", lambda x: max(f[0] for f in x) if x else float("-inf"))
    stats.register("max2", lambda x: max(f[1] for f in x) if x else float("-inf"))

    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=num_generations, stats=stats, halloffame=hof, verbose=True)

    with open(output_file, 'w') as f:
        f.write(f"{len(hof)}\n")
        for ind in hof:
            num_colors, neg_num_satisfied = ind.fitness.values
            f.write(f"{int(num_colors)} {-int(neg_num_satisfied)}\n")
            f.write(' '.join(str(color) for color in ind) + '\n')

if __name__ == "__main__":
    input_file = 'sample_5.txt'
    output_file = 'output_results_5.txt'
    run_nsga2(input_file, output_file)

gen	nevals	avg1  	avg2	min1  	min2	max1  	max2
0  	100   	999999	0   	999999	0   	999999	0   
1  	86    	989999	-0.02	10    	-2  	999999	0   
2  	76    	999999	0    	999999	0   	999999	0   
3  	80    	979999	-0.02	10    	-1  	999999	0   
4  	78    	989999	-0.01	10    	-1  	999999	0   
5  	83    	989999	-0.01	11    	-1  	999999	0   
6  	77    	999999	0    	999999	0   	999999	0   
7  	80    	989999	-0.01	11    	-1  	999999	0   
8  	72    	999999	0    	999999	0   	999999	0   
9  	79    	989999	0    	12    	0   	999999	0   
10 	76    	989999	0    	11    	0   	999999	0   
11 	86    	989999	-0.01	11    	-1  	999999	0   
12 	69    	989999	0    	11    	0   	999999	0   
13 	85    	989999	-0.01	10    	-1  	999999	0   
14 	71    	999999	0    	999999	0   	999999	0   
15 	72    	989999	-0.01	11    	-1  	999999	0   
16 	84    	999999	0    	999999	0   	999999	0   
17 	80    	989999	-0.02	11    	-2  	999999	0   
18 	81    	999999	0    	999999	0   	999999	0   
19 	81    	999999	0    	999999	0   	999999